In [95]:
# import module
from datetime import datetime
from datetime import timedelta
import requests
import os
import json
import Bearer
import pytz
import time


In [96]:
# 時刻を変換する関数

def make_local_dt (dt):
    utc_dt = dt - timedelta(hours=+9)
    return (pytz.utc.localize(utc_dt).astimezone(pytz.timezone("Asia/Tokyo")) )

    
def dt2localiso(dt):
    # この段階で
    if dt is None:
        return ''
    
    return dt.isoformat()

def iso_to_localdt(iso_str):
    dt = None
    try:
        dt = datetime.strptime(iso_str, '%Y-%m-%dT%H:%M:%S.%fZ')
        dt = pytz.utc.localize(dt).astimezone(pytz.timezone("Asia/Tokyo"))
    except ValueError:
        try:
            dt = datetime.strptime(iso_str, '%Y-%m-%dT%H:%M:%S.%f%z')
            dt = dt.astimezone(pytz.timezone("Asia/Tokyo"))
        except ValueError:
            pass
    return dt

In [97]:
movie_name

'2021-04-23_名探偵天国へのカウントダウン'

In [98]:
# 開始日
# start_time_dt = make_local_dt(datetime(2021, 3, 26, 21, 0, 0) )# バイオハザード
#start_time_dt = make_local_dt(datetime(2021, 4, 2, 21, 0, 0) )# 2021.04.02 ハウルの動く城
#movie_name = str(start_time_dt.date())+"_"+"ハウルの動く城"

# 2021.04.09 ゲド戦記
# start_time_dt = make_local_dt(datetime(2021, 4, 9, 21, 0, 0) )
# movie_name = str(start_time_dt.date())+"_"+"ゲド戦記"

# 2021.04.16 名探偵コナン異次元の狙撃手
# start_time_dt = make_local_dt(datetime(2021, 4, 16, 21, 0, 0) )
# movie_name = str(start_time_dt.date())+"_"+"名探偵コナン異次元の狙撃手"

# 2021.04.23 名探偵コナン天国へのカウントダウン
#start_time_dt = make_local_dt(datetime(2021, 4, 23, 21, 0, 0) )
#movie_name = str(start_time_dt.date())+"_"+"名探偵天国へのカウントダウン"

# 2021.04.30 るろうに剣心
start_time_dt = make_local_dt(datetime(2021, 4, 30, 21, 0, 0) )
movie_name = str(start_time_dt.date())+"_"+"るろうに剣心"

# 終了日
end_time_dt = start_time_dt + timedelta(hours=+3)

start_time = dt2localiso(start_time_dt)
end_time = dt2localiso(end_time_dt)

In [99]:
# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = Bearer.BEARER

search_url = "https://api.twitter.com/2/tweets/search/all"

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", search_url, headers=headers, params=params)
    print("response_st",response.status_code)
    if response.status_code != 200:
        raise Exception("response_st",response.status_code, response.text)
    return response.json()



In [100]:
# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {'query': '#金曜ロードSHOW',
                'tweet.fields': "created_at,author_id,public_metrics", # スペースをいれない
                'max_results' : 500,
                "start_time":start_time ,
               "end_time": end_time,
               }

json_response = None
query_params.pop("until_id", None)
tweet_data = []

for i in range(10):
    time.sleep(1) # 連続するとエラーになる
    if json_response:
        print(i)
        query_params.pop("start_time", None)
        query_params.pop("end_time", None)
        query_params["until_id"] = int(json_response["data"][-1]["id"])
        
        print(query_params["until_id"])
    
    
    json_response = connect_to_endpoint(search_url, 
                                    create_headers(bearer_token), # headerを作る
                                    query_params,
                                    )
    for i,tmp_content in enumerate(json_response["data"]):
        
        
        try:
            tweet_data.append([tmp_content["id"], 
                               tmp_content["created_at"], tmp_content["text"].replace('\n',''),
                               tmp_content["public_metrics"]["like_count"],
                               tmp_content["public_metrics"]["retweet_count"], 
                               tmp_content["author_id"] ])
        except Exception as e:
            print(e)
        
        
        if i % 10 == 0:
            created = iso_to_localdt(tmp_content["created_at"])
            print(tmp_content["id"], created  )
            
        if created < start_time_dt :
            break
    
    if created < start_time_dt :
        break
    

response_st 200
1388145688760918018 2021-04-30 23:58:17+09:00
1388136531429232640 2021-04-30 23:21:54+09:00
1388132016416202753 2021-04-30 23:03:57+09:00
1388130726101487617 2021-04-30 22:58:49+09:00
1388130160826675203 2021-04-30 22:56:35+09:00
1388129661675204612 2021-04-30 22:54:36+09:00
1388129466526801921 2021-04-30 22:53:49+09:00
1388129294468018177 2021-04-30 22:53:08+09:00
1388127245953208321 2021-04-30 22:45:00+09:00
1388123978292633601 2021-04-30 22:32:01+09:00
1388119596582924288 2021-04-30 22:14:36+09:00
1388115985454931969 2021-04-30 22:00:15+09:00
1388111629544353797 2021-04-30 21:42:57+09:00
1388108373917270017 2021-04-30 21:30:00+09:00
1388105012316835842 2021-04-30 21:16:39+09:00
1388103319067906052 2021-04-30 21:09:55+09:00
1388101974470447106 2021-04-30 21:04:35+09:00
1
1388101141066518529
response_st 200
1388100783623663619 2021-04-30 20:59:51+09:00


In [101]:
import pandas as pd
df = pd.DataFrame(tweet_data, columns = ["tweet_id", "date","text","like","retweet", "author_id"])
df.set_index("tweet_id").to_csv("./"+ movie_name +".csv")

In [102]:
# !pip install pymlask